# TradingView Screener를 활용한 주식 종목 스크리닝

이 노트북에서는 `tradingview-screener` 라이브러리를 사용하여 다양한 조건으로 주식 종목을 스크리닝합니다.

## 주요 기능
- 기본 스크리닝 (종가, 거래량, 시가총액)
- 거래량 기반 스크리닝
- 기술적 지표 스크리닝 (MACD, RSI)
- 모멘텀 스크리닝
- 가치 투자 스크리닝


## 1. 라이브러리 설치 및 임포트


In [4]:
# 필요한 라이브러리 설치
# !pip install tradingview-screener pandas matplotlib seaborn


In [5]:
# 라이브러리 임포트
from tradingview_screener import Query, col
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 설정 (맥OS)
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

# 판다스 출력 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', None)

# Seaborn 스타일 설정
sns.set_theme(style='whitegrid', palette='husl')

print("라이브러리 로딩 완료!")


라이브러리 로딩 완료!


## 2. 기본 스크리닝

가장 기본적인 스크리닝 - 종목명, 종가, 거래량, 시가총액 조회


In [6]:
# 기본 스크리닝: 상위 50개 종목
count, df_basic = (
    Query()
    .select('name', 'close', 'volume', 'market_cap_basic', 'change')
    .get_scanner_data()
)

print(f"전체 {count}개 종목 중 {len(df_basic)}개 조회됨")
df_basic.head(10)


전체 19256개 종목 중 50개 조회됨


,ticker,name,close,volume,market_cap_basic,change
0,AMEX:SPY,SPY,681.53,62953844,NaN,0.185221
1,NASDAQ:QQQ,QQQ,622.00,54541216,NaN,0.782604
2,NASDAQ:NVDA,NVDA,181.46,182625215,4.409478e+12,0.855936
3,NASDAQ:TSLA,TSLA,429.24,69331263,1.427575e+12,-0.209234
4,NASDAQ:AAPL,AAPL,286.19,53669494,4.228844e+12,1.091487
5,NASDAQ:GOOGL,GOOGL,315.81,35854612,3.812127e+12,0.292166
6,NASDAQ:AMZN,AMZN,234.42,45784575,2.506000e+12,0.230888
7,NASDAQ:MSFT,MSFT,490.00,19562722,3.641865e+12,0.669762
8,NASDAQ:AMD,AMD,215.24,42001820,3.504196e+11,-2.056789
9,NASDAQ:AVGO,AVGO,381.57,22206602,1.801913e+12,-1.168152


## 3. 거래량 기반 스크리닝

상대 거래량(10일 평균 대비)이 높은 종목을 찾습니다.


In [7]:
# 거래량 스크리닝: 상대 거래량 1.5배 이상
count, df_volume = (
    Query()
    .select('name', 'close', 'volume', 'relative_volume_10d_calc', 'change', 'market_cap_basic')
    .where(
        col('relative_volume_10d_calc') > 1.5,
        col('market_cap_basic') > 100_000_000  # 시가총액 1억 달러 이상
    )
    .order_by('relative_volume_10d_calc', ascending=False)
    .limit(30)
    .get_scanner_data()
)

print(f"상대 거래량 1.5배 이상: {count}개 종목 중 {len(df_volume)}개 조회")
df_volume


상대 거래량 1.5배 이상: 1299개 종목 중 30개 조회


,ticker,name,close,volume,relative_volume_10d_calc,change,market_cap_basic
0,OTC:DBOEF,DBOEF,263.8000,142877,332.194838,-0.151400,4.796445e+10
1,OTC:KPELY,KPELY,15.8400,348396,227.605671,-0.251889,1.385248e+10
2,OTC:LSRCF,LSRCF,170.6200,101211,162.509634,0.134984,1.538616e+10
3,OTC:STMEF,STMEF,23.5150,180008,127.619993,1.335919,2.098310e+10
4,OTC:HLPPY,HLPPY,5.8530,277689,105.229073,1.614583,5.854166e+09
5,OTC:GGDVY,GGDVY,49.2500,21694,105.157538,-1.024920,6.343817e+09
6,OTC:FRFXF,FRFXF,18.0000,86023,103.692141,1.838755,4.081744e+10
7,OTC:ZHEXF,ZHEXF,1.0040,928718,98.709479,1.414141,6.062267e+09
8,OTC:DISPF,DISPF,277.8500,2514,91.418182,-4.600858,3.012762e+10
9,OTC:DWLAF,DWLAF,1.0600,71750,89.788512,2.912621,1.389804e+09


## 4. 기술적 지표 스크리닝 (MACD, RSI)

MACD 골든 크로스와 RSI 조건을 활용한 스크리닝


In [8]:
# MACD 골든 크로스 스크리닝
count, df_macd = (
    Query()
    .select('name', 'close', 'change', 'volume', 'MACD.macd', 'MACD.signal', 'RSI')
    .where(
        col('market_cap_basic') > 50_000_000,
        col('MACD.macd') >= col('MACD.signal'),  # MACD가 시그널 위에 있음 (골든 크로스)
        col('RSI').between(30, 70)  # 과매수/과매도 아닌 구간
    )
    .order_by('volume', ascending=False)
    .limit(30)
    .get_scanner_data()
)

print(f"MACD 골든 크로스 종목: {count}개 중 {len(df_macd)}개 조회")
df_macd


MACD 골든 크로스 종목: 5339개 중 30개 조회


,ticker,name,close,change,volume,MACD.macd,MACD.signal,RSI
0,NASDAQ:BYND,BYND,1.2900,-3.731343,233167768,-0.166014,-0.230615,50.086889
1,NASDAQ:INTC,INTC,43.4700,8.647838,167195636,0.828437,0.189735,69.769342
2,NASDAQ:BITF,BITF,3.1000,-5.487805,93787257,-0.182109,-0.242096,47.206277
3,NASDAQ:ONDS,ONDS,8.0700,6.464380,90635527,0.259068,0.075703,55.172941
4,NASDAQ:TSLA,TSLA,429.2400,-0.209234,69331263,-2.581048,-4.602919,52.095014
5,NASDAQ:AAL,AAL,14.2400,2.005731,64579405,0.331858,0.199193,64.219356
6,NASDAQ:ASST,ASST,1.0350,-1.428571,64148284,-0.168194,-0.208491,39.164425
7,AMEX:BMNR,BMNR,31.9100,10.262612,60475192,-4.416138,-4.848890,43.095594
8,NYSE:BBAI,BBAI,5.7900,-4.297521,59959702,-0.177434,-0.214915,45.278495
9,NASDAQ:SOFI,SOFI,29.5100,1.653462,59744761,0.069753,-0.105990,54.496481


## 5. 모멘텀 스크리닝

상승 중이면서 거래량이 증가하는 종목 찾기


In [9]:
# 모멘텀 스크리닝
count, df_momentum = (
    Query()
    .select('name', 'close', 'change', 'volume', 'relative_volume_10d_calc', 
            'market_cap_basic', 'Perf.W', 'Perf.1M')  # 주간, 월간 수익률
    .where(
        col('change') > 3,  # 오늘 3% 이상 상승
        col('relative_volume_10d_calc') > 1.2,  # 거래량 1.2배 이상
        col('market_cap_basic') > 100_000_000  # 시가총액 1억 달러 이상
    )
    .order_by('change', ascending=False)
    .limit(25)
    .get_scanner_data()
)

print(f"모멘텀 상승 종목: {count}개 중 {len(df_momentum)}개 조회")
df_momentum


모멘텀 상승 종목: 289개 중 25개 조회


,ticker,name,close,change,volume,relative_volume_10d_calc,market_cap_basic,Perf.W,Perf.1M
0,OTC:AMVIF,AMVIF,42.5000,4.249990e+07,647,1.833381,4.252167e+09,4.249990e+07,3.658537
1,OTC:ACEHF,ACEHF,0.0323,2.230000e+02,10000,2.433327,5.520322e+08,2.230000e+02,822.857143
2,OTC:NFGRF,NFGRF,0.2127,9.934396e+01,5000,6.572893,1.832429e+08,9.934396e+01,99.343955
3,OTC:AGTEF,AGTEF,0.0202,9.423077e+01,20000,1.514211,2.322828e+08,4.975124e-01,-3.579952
4,NYSE:EB,EB,4.4300,7.862903e+01,29467518,71.746614,4.327009e+08,8.155738e+01,94.298246
5,OTC:BNPJY,BNPJY,2.5000,5.625000e+01,200,1.312336,1.476843e+09,5.625000e+01,56.250000
6,OTC:MKNGF,MKNGF,0.6000,5.584416e+01,19173,1.210455,2.083157e+08,8.108108e+00,-25.000000
7,OTC:TRGNF,TRGNF,1.0800,5.428571e+01,2000,4.257131,1.430077e+08,5.428571e+01,54.285714
8,OTC:COOSF,COOSF,13.9650,5.229008e+01,5878,4.826340,2.360411e+08,3.731563e+01,72.194821
9,OTC:MRTPY,MRTPY,7.4500,5.050505e+01,1167,2.147589,4.943515e+08,5.050505e+01,50.505051


## 6. 시가총액별 스크리닝


In [10]:
# 대형주 스크리닝 (시가총액 100억 달러 이상)
count, df_large_cap = (
    Query()
    .select('name', 'close', 'change', 'volume', 'market_cap_basic', 'sector')
    .where(
        col('market_cap_basic') > 10_000_000_000  # 100억 달러 이상
    )
    .order_by('market_cap_basic', ascending=False)
    .limit(30)
    .get_scanner_data()
)

print(f"대형주 종목: {count}개 중 {len(df_large_cap)}개 조회")
df_large_cap


대형주 종목: 2245개 중 30개 조회


,ticker,name,close,change,volume,market_cap_basic,sector
0,NASDAQ:NVDA,NVDA,181.4600,0.855936,182625215,4.409478e+12,Electronic Technology
1,NASDAQ:AAPL,AAPL,286.1900,1.091487,53669494,4.228844e+12,Electronic Technology
2,NASDAQ:GOOGL,GOOGL,315.8100,0.292166,35854612,3.812127e+12,Technology Services
3,NASDAQ:GOOG,GOOG,316.0200,0.285605,24668179,3.811877e+12,Technology Services
4,NASDAQ:MSFT,MSFT,490.0000,0.669762,19562722,3.641865e+12,Technology Services
5,NASDAQ:AMZN,AMZN,234.4200,0.230888,45784575,2.506000e+12,Retail Trade
6,NASDAQ:AVGO,AVGO,381.5700,-1.168152,22206602,1.801913e+12,Electronic Technology
7,NASDAQ:META,META,647.1000,0.972116,11640836,1.631033e+12,Technology Services
8,NASDAQ:TSLA,TSLA,429.2400,-0.209234,69331263,1.427575e+12,Consumer Durables
9,NYSE:TSM,TSM,292.0900,1.532953,10745439,1.162580e+12,Electronic Technology


In [11]:
# 중형주 스크리닝 (시가총액 10억 ~ 100억 달러)
count, df_mid_cap = (
    Query()
    .select('name', 'close', 'change', 'volume', 'market_cap_basic', 'sector')
    .where(
        col('market_cap_basic').between(1_000_000_000, 10_000_000_000)
    )
    .order_by('change', ascending=False)
    .limit(30)
    .get_scanner_data()
)

print(f"중형주 종목: {count}개 중 {len(df_mid_cap)}개 조회")
df_mid_cap


중형주 종목: 3130개 중 30개 조회


,ticker,name,close,change,volume,market_cap_basic,sector
0,OTC:AMVIF,AMVIF,42.50000,4.249990e+07,647,4.252167e+09,Producer Manufacturing
1,OTC:YRCCF,YRCCF,17.19000,1.951411e+02,400,1.177163e+09,Industrial Services
2,OTC:SHPHF,SHPHF,0.20770,1.077000e+02,106,1.937841e+09,Health Technology
3,OTC:LGBOF,LGBOF,1.95000,9.500000e+01,100,2.509603e+09,Non-Energy Minerals
4,OTC:MGHTF,MGHTF,3.90000,9.259259e+01,1500,5.527833e+09,Utilities
5,OTC:UNEGF,UNEGF,0.07000,7.283951e+01,10000,1.818850e+09,Energy Minerals
6,OTC:ALTB,ALTB,8.00000,6.000000e+01,100,7.968800e+09,Retail Trade
7,OTC:BNPJY,BNPJY,2.50000,5.625000e+01,200,1.476843e+09,Energy Minerals
8,OTC:PUPOF,PUPOF,18.39000,5.427464e+01,1690,6.790875e+09,Utilities
9,OTC:BUMTF,BUMTF,1.15000,5.333333e+01,2812,1.994266e+09,Process Industries


## 7. 가치 투자 스크리닝

낮은 P/E 비율과 배당 수익률 기준 스크리닝


In [12]:
# 가치 투자 스크리닝
count, df_value = (
    Query()
    .select('name', 'close', 'price_earnings_ttm', 'dividend_yield_recent', 
            'price_book_ratio', 'market_cap_basic', 'sector')
    .where(
        col('price_earnings_ttm').between(5, 15),  # P/E 5~15
        col('dividend_yield_recent') > 2,  # 배당수익률 2% 이상
        col('market_cap_basic') > 1_000_000_000  # 시가총액 10억 달러 이상
    )
    .order_by('dividend_yield_recent', ascending=False)
    .limit(25)
    .get_scanner_data()
)

print(f"가치 투자 종목: {count}개 중 {len(df_value)}개 조회")
df_value


가치 투자 종목: 1024개 중 25개 조회


,ticker,name,close,price_earnings_ttm,dividend_yield_recent,price_book_ratio,market_cap_basic,sector
0,OTC:AKRYY,AKRYY,5.7000,5.795628,54.533108,1.376659,1.467902e+09,Utilities
1,OTC:MBRFY,MBRFY,3.4600,6.229744,26.149770,6.677474,4.889898e+09,Consumer Non-Durables
2,NYSE:ORC,ORC,7.2100,14.875181,19.972261,0.891111,1.089293e+09,Finance
3,OTC:UNEGF,UNEGF,0.0700,10.949527,18.181818,1.050753,1.818850e+09,Energy Minerals
4,NYSE:GOF,GOF,12.2000,12.808399,17.909836,1.067537,1.987849e+09,Miscellaneous
5,AMEX:CLM,CLM,8.2800,8.625000,17.608696,1.177833,2.092828e+09,Miscellaneous
6,NYSE:EC,EC,9.9100,7.508713,16.891040,1.072388,2.041403e+10,Energy Minerals
7,NYSE:MNR,MNR,12.0800,12.584644,16.059603,1.192182,2.034547e+09,Energy Minerals
8,NYSE:IIPR,IIPR,49.8700,11.789319,15.239623,0.738804,1.398066e+09,Finance
9,NYSE:IIPR/PA,IIPR/PA,24.4400,5.777641,15.239623,0.362069,1.398066e+09,Finance


## 8. 사용자 정의 스크리닝 함수


In [13]:
def custom_screener(
    min_market_cap: int = 100_000_000,
    max_market_cap: int = None,
    min_volume: int = 100_000,
    min_change: float = None,
    max_change: float = None,
    min_rel_volume: float = None,
    min_rsi: float = None,
    max_rsi: float = None,
    macd_bullish: bool = False,
    sort_by: str = 'volume',
    ascending: bool = False,
    limit: int = 50
):
    """
    사용자 정의 스크리닝 함수
    
    Parameters:
    -----------
    min_market_cap : int - 최소 시가총액 (달러)
    max_market_cap : int - 최대 시가총액 (달러)
    min_volume : int - 최소 거래량
    min_change : float - 최소 변동률 (%)
    max_change : float - 최대 변동률 (%)
    min_rel_volume : float - 최소 상대 거래량 배수
    min_rsi : float - 최소 RSI
    max_rsi : float - 최대 RSI
    macd_bullish : bool - MACD 골든크로스 필터 적용 여부
    sort_by : str - 정렬 기준 컬럼
    ascending : bool - 오름차순 정렬 여부
    limit : int - 조회할 종목 수
    
    Returns:
    --------
    pandas.DataFrame - 스크리닝 결과
    """
    
    # 조건 목록 생성
    conditions = []
    
    # 시가총액 조건
    if max_market_cap:
        conditions.append(col('market_cap_basic').between(min_market_cap, max_market_cap))
    else:
        conditions.append(col('market_cap_basic') > min_market_cap)
    
    # 거래량 조건
    conditions.append(col('volume') > min_volume)
    
    # 변동률 조건
    if min_change is not None and max_change is not None:
        conditions.append(col('change').between(min_change, max_change))
    elif min_change is not None:
        conditions.append(col('change') > min_change)
    elif max_change is not None:
        conditions.append(col('change') < max_change)
    
    # 상대 거래량 조건
    if min_rel_volume is not None:
        conditions.append(col('relative_volume_10d_calc') > min_rel_volume)
    
    # RSI 조건
    if min_rsi is not None and max_rsi is not None:
        conditions.append(col('RSI').between(min_rsi, max_rsi))
    elif min_rsi is not None:
        conditions.append(col('RSI') > min_rsi)
    elif max_rsi is not None:
        conditions.append(col('RSI') < max_rsi)
    
    # MACD 조건
    if macd_bullish:
        conditions.append(col('MACD.macd') >= col('MACD.signal'))
    
    # 쿼리 실행
    query = (
        Query()
        .select('name', 'close', 'change', 'volume', 'relative_volume_10d_calc',
                'market_cap_basic', 'RSI', 'MACD.macd', 'MACD.signal', 'sector')
        .where(*conditions)
        .order_by(sort_by, ascending=ascending)
        .limit(limit)
    )
    
    count, df = query.get_scanner_data()
    
    print(f"조건에 맞는 종목: {count}개 중 {len(df)}개 조회")
    
    return df


In [14]:
# 사용자 정의 스크리닝 예시
# 조건: 중형주, 2% 이상 상승, 거래량 증가, MACD 골든크로스

df_custom = custom_screener(
    min_market_cap=500_000_000,
    max_market_cap=10_000_000_000,
    min_volume=500_000,
    min_change=2.0,
    min_rel_volume=1.2,
    macd_bullish=True,
    sort_by='change',
    ascending=False,
    limit=20
)

df_custom


조건에 맞는 종목: 64개 중 20개 조회


,ticker,name,close,change,volume,relative_volume_10d_calc,market_cap_basic,RSI,MACD.macd,MACD.signal,sector
0,NASDAQ:FBYD,FBYD,20.80,20.440069,732481,10.638362,2.518090e+09,69.851372,1.724212,0.962943,Industrial Services
1,NASDAQ:TMC,TMC,7.40,17.740652,23049597,2.272952,3.059841e+09,61.835256,-0.085690,-0.346700,Non-Energy Minerals
2,NASDAQ:VERA,VERA,37.25,12.878788,4354087,3.183776,2.381320e+09,71.757235,1.507663,0.663915,Health Technology
3,NYSE:EVEX,EVEX,4.67,11.990408,3083723,2.642696,1.626582e+09,64.071311,-0.052899,-0.117483,Electronic Technology
4,NYSE:CTRI,CTRI,24.46,10.229833,4281515,2.248378,2.435712e+09,70.021610,0.476190,0.126424,Utilities
5,NYSE:CSAN,CSAN,5.34,8.757637,2654931,1.827894,4.535634e+09,65.777467,0.098139,0.065678,Utilities
6,NYSE:ADCT,ADCT,4.60,8.490566,1309400,1.917060,5.698347e+08,64.090673,0.074329,0.034000,Health Technology
7,NASDAQ:SAIA,SAIA,314.50,7.282961,973906,2.293188,8.379110e+09,66.779360,0.312366,-4.967937,Transportation
8,NASDAQ:OMCL,OMCL,38.46,6.478405,622874,1.668729,1.725951e+09,69.578930,1.175447,1.089960,Technology Services
9,NASDAQ:TRON,TRON,2.00,6.382979,1453338,1.293673,5.142308e+08,43.973209,-0.219364,-0.262171,Consumer Durables


## 9. 결과 저장


In [15]:
# 결과를 CSV로 저장
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# 예시: 모멘텀 스크리닝 결과 저장
if 'df_momentum' in dir() and not df_momentum.empty:
    filename = f'momentum_screening_{timestamp}.csv'
    df_momentum.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"저장 완료: {filename}")


저장 완료: momentum_screening_20251203_174517.csv


## 사용 가능한 필드 목록 (예시)

| 필드명 | 설명 |
|--------|------|
| name | 종목명 |
| close | 종가 |
| open | 시가 |
| high | 고가 |
| low | 저가 |
| volume | 거래량 |
| change | 변동률 (%) |
| market_cap_basic | 시가총액 |
| relative_volume_10d_calc | 10일 평균 대비 상대 거래량 |
| RSI | RSI (14일) |
| MACD.macd | MACD 라인 |
| MACD.signal | MACD 시그널 |
| price_earnings_ttm | P/E 비율 (TTM) |
| dividend_yield_recent | 배당 수익률 |
| price_book_ratio | P/B 비율 |
| sector | 섹터 |
| industry | 산업 |
| Perf.W | 주간 수익률 |
| Perf.1M | 월간 수익률 |
| price_52_week_high | 52주 최고가 |
| price_52_week_low | 52주 최저가 |


---

# Part 2: Global Screener - 4가지 투자 전략

`global_screener_spec.md`에 정의된 4가지 투자 전략 스크리너입니다:
1. **Cyclical (경기민감형)** - 저 PBR, 저 EV/EBITDA
2. **Growth (고성장형)** - 높은 매출 성장률, 저 PEG
3. **Finance (금융/자산주)** - 극저 PBR, 높은 ROE, 배당
4. **Defensive (경기방어주)** - 안정적 영업이익률, FCF, 배당


In [135]:
# 공통 설정

# 기술 등급 기준 (Technical Rating)
# Recommend.All: -1(Strong Sell) ~ 1(Strong Buy)
# -1 ~ -0.5: Strong Sell, -0.5 ~ -0.1: Sell, -0.1 ~ 0.1: Neutral
# 0.1 ~ 0.5: Buy, 0.5 ~ 1: Strong Buy
TECH_RATING_BUY = 0.1
TECH_RATING_STRONG_BUY = 0.5

# 애널리스트 평점 기준 (Analyst Rating)
# 가중 평균 점수 (-2 ~ 2 스케일)
# Strong Buy=2, Buy=1, Hold=0, Sell=-1, Strong Sell=-2
ANALYST_SCORE_BUY = 0.5        # Buy 이상 기준
ANALYST_SCORE_STRONG_BUY = 1.0  # Strong Buy 기준

# 최소 애널리스트 수 (신뢰도 기준)
# 3명 이상: 유연한 기준 (소형주 포함)
MIN_ANALYST_COUNT = 3

# 섹터 매핑 (TradingView 영문 섹터명)
SECTORS = {
    'cyclical': [
        'Process Industries', 'Non-Energy Minerals', 'Producer Manufacturing',
        'Consumer Durables', 'Energy Minerals', 'Electronic Technology',
    ],
    'growth': [
        'Technology Services', 'Health Services', 'Commercial Services', 'Health Technology',
    ],
    'finance': ['Finance'],
    'defensive': ['Consumer Non-Durables', 'Utilities', 'Communications'],
}


def calculate_analyst_score(df):
    """
    애널리스트 평점 계산 함수
    점수 = (2×strong_buy + 1×buy + 0×hold - 1×sell - 2×strong_sell) / total
    """
    if df.empty:
        return df
    
    required_cols = ['recommendation_buy', 'recommendation_over', 'recommendation_hold',
                     'recommendation_under', 'recommendation_sell', 'recommendation_total']
    
    if not all(c in df.columns for c in required_cols):
        return df
    
    for c in required_cols:
        df[c] = df[c].fillna(0)
    
    df['analyst_score'] = (
        2 * df['recommendation_buy'] +
        1 * df['recommendation_over'] +
        0 * df['recommendation_hold'] +
        -1 * df['recommendation_under'] +
        -2 * df['recommendation_sell']
    ) / df['recommendation_total'].replace(0, 1)
    
    def get_rating(score, total):
        if pd.isna(score) or total == 0:
            return 'N/A'
        elif score >= 1.0:
            return 'Strong Buy'
        elif score >= 0.5:
            return 'Buy'
        elif score >= -0.5:
            return 'Hold'
        elif score >= -1.0:
            return 'Sell'
        else:
            return 'Strong Sell'
    
    df['analyst_rating'] = df.apply(lambda r: get_rating(r['analyst_score'], r['recommendation_total']), axis=1)
    return df


def filter_by_analyst(df, min_score=ANALYST_SCORE_BUY):
    """애널리스트 평점으로 필터링"""
    if df.empty or 'analyst_score' not in df.columns:
        return df
    return df[df['analyst_score'] >= min_score]


print("공통 설정 완료!")


공통 설정 완료!


## 전략 1: Cyclical (경기민감형)

**목표:** 자산 가치 대비 저평가되고, 현금 창출력이 좋은 기업

| 필터 | 조건 | 설명 |
|------|------|------|
| PBR | < 1 | 자산가치 대비 저평가 |
| EV/EBITDA | < 6 | 현금 창출력 대비 저평가 |
| 유동비율 | >= 1.5 | 경기 침체 시 버틸 현금 체력 |
| 애널리스트/기술 등급 | Buy 이상 | 전문가 평가 매수 이상 |


In [149]:
# Cyclical (경기민감형) 스크리너
count_cyclical, df_cyclical = (
    Query()
    .select(
        'name', 'close', 'change', 'volume', 'market_cap_basic',
        'sector', 'industry',
        'price_book_fq',               # PBR
        'enterprise_value_ebitda_ttm', # EV/EBITDA
        'current_ratio_fq',            # 유동비율
        'recommendation_mark',         # 애널리스트 평점 (문자열)
        'recommendation_hold',
        'recommendation_sell',
        'recommendation_buy',
        'recommendation_total',
        'recommendation_over',
        'recommendation_under',
        'Recommend.All',               # 기술 등급 (숫자)
    )
    .where(
        col('is_primary') == True,
        col('price_book_fq') < 1,
        col('price_book_fq') > 0,
        col('enterprise_value_ebitda_ttm') < 6,
        col('enterprise_value_ebitda_ttm') > 0,
        col('current_ratio_fq') >= 1.5,
        col('recommendation_total') >= MIN_ANALYST_COUNT,  # 최소 애널리스트 수
        col('Recommend.All') >= TECH_RATING_BUY,
    )
    .order_by('enterprise_value_ebitda_ttm', ascending=True)
    .limit(1000)
    .get_scanner_data()
)

# 애널리스트 점수 계산
df_cyclical = calculate_analyst_score(df_cyclical)

# 애널리스트 평점 Buy 이상 필터링
df_cyclical = filter_by_analyst(df_cyclical, ANALYST_SCORE_BUY)

# 섹터 필터링
if not df_cyclical.empty and 'sector' in df_cyclical.columns:
    df_cyclical_filtered = df_cyclical[df_cyclical['sector'].isin(SECTORS['cyclical'])]

print(f"📊 Cyclical 종목: {count_cyclical}개 중 {len(df_cyclical_filtered)}개 필터링됨")
print(f"조건: 기술등급 Buy↑, 애널리스트 Buy↑ (score >= 0.5)")
df_cyclical_filtered[['name', 'close', 'price_book_fq', 'enterprise_value_ebitda_ttm', 
             'current_ratio_fq', 'Recommend.All', 'analyst_score', 'analyst_rating', 'recommendation_total']]


📊 Cyclical 종목: 9개 중 0개 필터링됨
조건: 기술등급 Buy↑, 애널리스트 Buy↑ (score >= 0.5)


,name,close,price_book_fq,enterprise_value_ebitda_ttm,current_ratio_fq,Recommend.All,analyst_score,analyst_rating,recommendation_total


## 전략 2: Growth (고성장형)

**목표:** 매출이 빠르게 늘면서, 성장성 대비 주가가 싼 기업

| 필터 | 조건 | 설명 |
|------|------|------|
| 매출 성장률 YoY | >= 20% | 전년 대비 고속 성장 |
| PEG 비율 | < 1 | 성장률 감안 시 저평가 |
| 부채비율 | < 150% | 금리 리스크 관리 |
| 애널리스트/기술 등급 | Buy 이상 | 전문가 평가 매수 이상 |


In [156]:
# Growth (고성장형) 스크리너
count_growth, df_growth = (
    Query()
    .select(
        'name', 'close', 'change', 'volume', 'market_cap_basic',
        'sector', 'industry',
        'total_revenue_yoy_growth_ttm',  # 매출 성장률 YoY
        'price_earnings_growth_ttm',     # PEG 비율
        'debt_to_equity_fq',             # 부채비율
        'recommendation_buy',
        'recommendation_over',
        'recommendation_hold',
        'recommendation_under',
        'recommendation_sell',
        'recommendation_total',
        'Recommend.All',                 # 기술 등급 (숫자)
        'earnings_per_share_diluted_yoy_growth_ttm', # EPS 성장률
    )
    .where(
        col('is_primary') == True,
        col('total_revenue_yoy_growth_ttm') >= 20,
        col('price_earnings_growth_ttm') < 1,
        col('price_earnings_growth_ttm') >= 0.1,
        col('debt_to_equity_fq') < 1.5,
        col('recommendation_total') >= MIN_ANALYST_COUNT,  # 최소 애널리스트 수
        col('Recommend.All') >= TECH_RATING_BUY,
    )
    .order_by('price_earnings_growth_ttm', ascending=True)
    .limit(1000)
    .get_scanner_data()
)

# 애널리스트 점수 계산
df_growth = calculate_analyst_score(df_growth)

# 애널리스트 평점 Buy 이상 필터링
df_growth = filter_by_analyst(df_growth, ANALYST_SCORE_BUY)

# 섹터 필터링
if not df_growth.empty and 'sector' in df_growth.columns:
    df_growth_filtered = df_growth[df_growth['sector'].isin(SECTORS['growth'])]
else:
    df_growth_filtered = df_growth

print(f"📊 Growth 종목: {count_growth}개 중 {len(df_growth_filtered)}개 필터링됨")
print(f"조건: 기술등급 Buy↑, 애널리스트 Buy↑ (score >= 0.5)")
df_growth_filtered[['name', 'close', 'total_revenue_yoy_growth_ttm', 'price_earnings_growth_ttm',
           'Recommend.All', 'analyst_score', 'analyst_rating', 'recommendation_total']]


📊 Growth 종목: 40개 중 9개 필터링됨
조건: 기술등급 Buy↑, 애널리스트 Buy↑ (score >= 0.5)


,name,close,total_revenue_yoy_growth_ttm,price_earnings_growth_ttm,Recommend.All,analyst_score,analyst_rating,recommendation_total
0,ADMA,19.72,27.624850,0.108595,0.400000,2.000000,Strong Buy,4
3,KRYS,217.46,54.507739,0.116387,0.603030,1.833333,Strong Buy,12
4,GDYN,9.49,23.631829,0.117798,0.424242,2.000000,Strong Buy,6
6,SEZL,64.44,88.738124,0.175563,0.112121,1.500000,Strong Buy,4
8,HRMY,37.00,21.127649,0.225383,0.603030,1.545455,Strong Buy,11
12,CPRX,23.10,25.563214,0.285656,0.512121,1.857143,Strong Buy,7
17,TMDX,140.10,41.203720,0.346565,0.445455,1.461538,Strong Buy,13
25,SEI,53.25,92.331719,0.524295,0.603030,1.833333,Strong Buy,12
37,INOD,59.65,73.565611,0.958706,0.112121,1.800000,Strong Buy,5


## 전략 3: Finance (금융/자산주)

**목표:** 극도로 저평가된 자산과 높은 자본효율, 배당 매력

| 필터 | 조건 | 설명 |
|------|------|------|
| PBR | < 0.6 | 절대적 저평가 영역 |
| ROE | >= 10% | 저평가지만 돈은 잘 버는 곳 |
| 배당수익률 | >= 4% | 확실한 현금 보상 |
| 애널리스트/기술 등급 | Buy 이상 | 전문가 평가 매수 이상 |


In [155]:
# Finance (금융/자산주) 스크리너
count_finance, df_finance = (
    Query()
    .select(
        'name', 'close', 'change', 'volume', 'market_cap_basic',
        'sector', 'industry',
        'price_book_fq',              # PBR
        'return_on_equity_fq',        # ROE
        'dividend_yield_recent',      # 배당수익률
        'recommendation_buy',
        'recommendation_over',
        'recommendation_hold',
        'recommendation_under',
        'recommendation_sell',
        'recommendation_total',
        'Recommend.All',              # 기술 등급 (숫자)
    )
    .where(
        col('is_primary') == True,
        col('price_book_fq') < 0.6,
        col('price_book_fq') > 0,
        col('return_on_equity_fq') >= 10,
        col('dividend_yield_recent') >= 4,
        col('recommendation_total') >= MIN_ANALYST_COUNT,  # 최소 애널리스트 수
        col('Recommend.All') >= TECH_RATING_BUY,
    )
    .order_by('dividend_yield_recent', ascending=False)
    .limit(1000)
    .get_scanner_data()
)

# 애널리스트 점수 계산
df_finance = calculate_analyst_score(df_finance)

# 애널리스트 평점 Buy 이상 필터링
df_finance = filter_by_analyst(df_finance, ANALYST_SCORE_BUY)

# 섹터 필터링 (Finance만)
if not df_finance.empty and 'sector' in df_finance.columns:
    df_finance_filtered = df_finance[df_finance['sector'].isin(SECTORS['finance'])]
else:
    df_finance_filtered = df_finance

print(f"📊 Finance 종목: {count_finance}개 중 {len(df_finance_filtered)}개 필터링됨")
print(f"조건: 기술등급 Buy↑, 애널리스트 Buy↑ (score >= 0.5)")
df_finance_filtered[['name', 'close', 'price_book_fq', 'return_on_equity_fq', 'dividend_yield_recent',
            'Recommend.All', 'analyst_score', 'analyst_rating', 'recommendation_total']]


📊 Finance 종목: 1개 중 0개 필터링됨
조건: 기술등급 Buy↑, 애널리스트 Buy↑ (score >= 0.5)


,name,close,price_book_fq,return_on_equity_fq,dividend_yield_recent,Recommend.All,analyst_score,analyst_rating,recommendation_total


## 전략 4: Defensive (경기방어주)

**목표:** 마진이 안정적이고, 현금이 잘 돌며 배당을 주는 기업

| 필터 | 조건 | 설명 |
|------|------|------|
| 영업이익률 | >= 5% | 안정적인 마진 확보 |
| FCF | > 0 | 현금이 플러스인지 확인 |
| 배당수익률 | >= 3% | 은행 이자 이상의 수익 |
| 애널리스트/기술 등급 | Buy 이상 | 전문가 평가 매수 이상 |


In [157]:
# Defensive (경기방어주) 스크리너
count_defensive, df_defensive = (
    Query()
    .select(
        'name', 'close', 'change', 'volume', 'market_cap_basic',
        'sector', 'industry',
        'operating_margin_ttm',       # 영업이익률
        'free_cash_flow_ttm',         # 잉여현금흐름
        'dividend_yield_recent',      # 배당수익률
        'recommendation_buy',
        'recommendation_over',
        'recommendation_hold',
        'recommendation_under',
        'recommendation_sell',
        'recommendation_total',
        'Recommend.All',              # 기술 등급 (숫자)
    )
    .where(
        col('is_primary') == True,
        col('operating_margin_ttm') >= 5,
        col('free_cash_flow_ttm') > 0,
        col('dividend_yield_recent') >= 3,
        col('recommendation_total') >= MIN_ANALYST_COUNT,  # 최소 애널리스트 수
        col('Recommend.All') >= TECH_RATING_BUY,
    )
    .order_by('dividend_yield_recent', ascending=False)
    .limit(1000)
    .get_scanner_data()
)

# 애널리스트 점수 계산
df_defensive = calculate_analyst_score(df_defensive)

# 애널리스트 평점 Buy 이상 필터링
df_defensive = filter_by_analyst(df_defensive, ANALYST_SCORE_BUY)

# 섹터 필터링
if not df_defensive.empty and 'sector' in df_defensive.columns:
    df_defensive_filtered = df_defensive[df_defensive['sector'].isin(SECTORS['defensive'])]
else:
    df_defensive_filtered = df_defensive

print(f"📊 Defensive 종목: {count_defensive}개 중 {len(df_defensive_filtered)}개 필터링됨")
print(f"조건: 기술등급 Buy↑, 애널리스트 Buy↑ (score >= 0.5)")
df_defensive_filtered[['name', 'close', 'operating_margin_ttm', 'dividend_yield_recent',
              'Recommend.All', 'analyst_score', 'analyst_rating', 'recommendation_total']]


📊 Defensive 종목: 279개 중 6개 필터링됨
조건: 기술등급 Buy↑, 애널리스트 Buy↑ (score >= 0.5)


,name,close,operating_margin_ttm,dividend_yield_recent,Recommend.All,analyst_score,analyst_rating,recommendation_total
100,TIGO,52.36,24.705041,5.704506,0.175758,0.555556,Buy,9
128,BIP,36.41,24.545863,4.775125,0.603030,1.500000,Strong Buy,12
165,ELP,10.94,17.313574,4.081633,0.466667,2.000000,Strong Buy,10
189,REYN,24.42,13.403452,3.789127,0.245455,0.750000,Buy,8
233,IRDM,17.13,26.745821,3.413174,0.157576,1.111111,Strong Buy,9
252,KDP,28.33,21.979721,3.270530,0.133333,1.277778,Strong Buy,18


## 스크리닝 결과 요약


In [160]:
# 결과 요약
print("=" * 60)
print("📊 4가지 투자 전략 스크리닝 결과 요약")
print("=" * 60)
print(f"  • Cyclical (경기민감형): {len(df_cyclical_filtered)}개 종목")
print(f"  • Growth (고성장형): {len(df_growth_filtered)}개 종목")
print(f"  • Finance (금융/자산주): {len(df_finance_filtered)}개 종목")
print(f"  • Defensive (경기방어주): {len(df_defensive_filtered)}개 종목")
print("=" * 60)

# 결과 저장 함수
def save_all_results():
    from datetime import datetime
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    results = {
        'cyclical': df_cyclical_filtered,
        'growth': df_growth_filtered,
        'finance': df_finance_filtered,
        'defensive': df_defensive_filtered
    }
    
    for strategy, df in results.items():
        if not df.empty:
            filename = f'global_{strategy}_{timestamp}.csv'
            df.to_csv(filename, index=False, encoding='utf-8-sig')
            print(f"✅ 저장: {filename}")

# 저장하려면 아래 주석 해제
save_all_results()


📊 4가지 투자 전략 스크리닝 결과 요약
  • Cyclical (경기민감형): 0개 종목
  • Growth (고성장형): 9개 종목
  • Finance (금융/자산주): 0개 종목
  • Defensive (경기방어주): 6개 종목
✅ 저장: global_growth_20251204_145534.csv
✅ 저장: global_defensive_20251204_145534.csv
